In [2]:
import polars as pl
from pathlib import Path
from datasets import load_dataset
from tqdm.auto import tqdm
import torch

d:\anaconda3\envs\power_of_noise\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# golden_dataset = load_dataset('florin-hf/nq_open_gold')
# train_golden_dataset = golden_dataset['train']

train_golden_dataset = pl.read_parquet('data/embeddings_nq_gold.parquet')
nq_embeddings = train_golden_dataset['Embeddings'].to_list()

In [ ]:
# df = pl.read_parquet("data/corpus.parquet")

# # Tolgo gold
# idx_list = train_golden_dataset["idx_gold_in_corpus"]
# no_gold_df = df.with_row_index("row_num").filter(pl.col("row_num").is_in(idx_list).not_())

# sampled_corpus_df = no_gold_df.sample(n=1000000)

sampled_corpus_df = pl.read_parquet('data/embeddings_corpus_1000000.parquet')
embeddings = sampled_corpus_df['Embeddings'].to_list()

In [12]:
from sentence_transformers import SentenceTransformer
from joblib import Parallel, delayed
import os

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
sbert_model = SentenceTransformer("all-MiniLM-L6-v2", device=device)

cuda


In [13]:
random_dataset = []
gold_document_position = 7 # @param {type:"slider", min:0, max:7, step:1}
num_documents = 8
num_random_documents = num_documents - 1

In [6]:
text_list = sampled_corpus_df["Text"].to_list()
embeddings = []

# n_jobs = os.cpu_count() - 1
# embeddings = Parallel(n_jobs=n_jobs, verbose=10)(delayed(sbert_model.encode)(text) for text in text_list)

for text in tqdm(text_list):
    embeddings.append(sbert_model.encode(text))

# embeddings = sbert_model.encode(text_list, batch_size=128)

100%|██████████| 1000000/1000000 [2:11:38<00:00, 126.61it/s] 


In [25]:
import numpy as np

embeddings_series = pl.Series("Embeddings", embeddings)
sampled_corpus_df = sampled_corpus_df.with_columns(embeddings_series)
sampled_corpus_df.write_parquet('data/embeddings_corpus_1000000.parquet')

# all_embeddings = np.array(embeddings)
# np.save('data/embeddings.npy', all_embeddings)

In [8]:
nq_text_list = train_golden_dataset["text"]
nq_embeddings = []

# n_jobs = os.cpu_count() - 1
# nq_embeddings = Parallel(n_jobs=n_jobs, verbose=10)(delayed(sbert_model.encode)(text) for text in nq_text_list)

for text in tqdm(nq_text_list):
    nq_embeddings.append(sbert_model.encode(text))

100%|██████████| 72209/72209 [09:27<00:00, 127.33it/s]


In [22]:
import numpy as np

train_golden_dataset = train_golden_dataset.add_column("Embeddings", nq_embeddings)
train_golden_dataset.to_parquet('data/embeddings_nq_gold.parquet')

# all_embeddings = np.array(nq_embeddings)
# np.save('data/nq_embeddings.npy', all_embeddings)

Creating parquet from Arrow format: 100%|██████████| 73/73 [00:01<00:00, 41.57ba/s]


158415051

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
from sortedcontainers import SortedList

def top_k_indices_per_row(matrix, k):
    # Get the indices that would sort each row
    sorted_indices = np.argsort(matrix, axis=1)[:, ::-1]

    # Take the first 7 indices from each row (corresponding to the highest values)
    top_k_indices = sorted_indices[:, :k]

    return top_k_indices

def top_k_cosine_similarity_indexes(gold, documents = embeddings, top_K=7):
    similarity = cosine_similarity([gold], documents)
    idx_list = top_k_indices_per_row(similarity, top_K)
    return idx_list
        

In [25]:
import multiprocessing as mp
from datetime import datetime
from math import pow, sqrt

class SimilarityMetric():
	def __init__(self, processes):
		self.processes = processes
		pass

	#COSINE SIMILARITY
	
	#serial cosine similarity
	def square_rooted(self, x):
		return round(sqrt(sum([a*a for a in x])),3)
	
	def serial_cosine_similarity(self,x,y):
		numerator = sum(a*b for a,b in zip(x,y))
		denominator = self.square_rooted(x)*self.square_rooted(y)
		return round(numerator/float(denominator),3)
	
	def multplierr(self,a,b):
			return a*b
	
	#parallel cosine similarity
	def parallel_cosine_similarity(self,x,y):

		pool = mp.Pool(processes= self.processes)
		s = datetime.now()
		nums = pool.starmap(self.multplierr, zip(x,y))
		numerator = sum(nums)
		
		#x_sqr = pool.starmap( self.multplierr, zip(x,x))
		#y_sqr = pool.starmap( self.multplierr, zip(y,y))
		
		#denominator = round(sqrt(sum(x_sqr))) * round(sqrt(sum(y_sqr)))
		denominator = self.square_rooted(x)*self.square_rooted(y)
		
		e = datetime.now()
		print("Parallel Cosine Exec Time: ", e-s)
		return round(numerator/float(denominator),3)

In [ ]:
sm = SimilarityMetric(11)

half = len(nq_embeddings)//2
gold_embeddings_first_half = nq_embeddings[:half]
# gold_embeddings_second_half = nq_embeddings[half:]
first_half_similarity = sm.parallel_cosine_similarity(gold_embeddings_first_half, embeddings)

In [ ]:
n_jobs = os.cpu_count() - 2
half = len(nq_embeddings)//2
gold_embeddings_first_half = nq_embeddings[:half]
# gold_embeddings_second_half = nq_embeddings[half:]
print(len(gold_embeddings_first_half))
list_top_k_indexes = Parallel(n_jobs=n_jobs, verbose=10)(delayed(top_k_cosine_similarity_indexes)(emb, top_K=num_random_documents) for emb in gold_embeddings_first_half)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Unable to allocate 315 GiB
# similarity_matrix = cosine_similarity(nq_embeddings, embeddings)

# print(cosine_similarity(nq_embeddings[0].reshape(1, -1), nq_embeddings[0].reshape(1, -1)))

# 32 ore
similarity_matrix = []
for embedding in tqdm(nq_embeddings):
    embedding_similarity = cosine_similarity([embedding], embeddings)
    similarity_matrix.append(embedding_similarity)

# print(len(similarity_matrix))
# Trova i 7 indici con le similarità più alte per ogni embedding di partenza
# top_k = 7
# top_k_indices = np.argsort(-similarity_matrix, axis=1)[:, :top_k]  # Ordina in ordine decrescente

# # Stampa i risultati
# for i, query in enumerate(nq_embeddings_test):
#     print(f"Query embedding {i}:")
#     print(f"Indices of 7 most similar embeddings: {top_k_indices[i]}")
#     print(f"Similarity scores: {similarity_matrix[i, top_k_indices[i]]}")
#     print()

In [53]:
for idx in range(len(train_golden_dataset)):
    question = {}
    question['Question'] = train_golden_dataset[idx]["question"]
    idx_gold_in_corpus = train_golden_dataset[idx]["idx_gold_in_corpus"]
    question['Answers'] = train_golden_dataset[idx]["answers"]
    question['Golden_idx'] = gold_document_position
    corpus_element = df.row(idx_gold_in_corpus)
    sampled_elements = df.sample(n=num_random_documents)
    question['Documents'] = [None] * (num_documents)
    
    # Insert Gold Document
    document = {}
    document['Title'] = corpus_element[0]
    document['Text'] = corpus_element[1]
    question['Documents'][gold_document_position] = document
    # Insert other random Documents
    df_idx = 0
    for i in range(len(question['Documents'])):
        document = {}
        if i == gold_document_position:
            continue
        document["Title"] = sampled_elements[df_idx]["Title"][0]
        document["Text"] = sampled_elements[df_idx]["Text"][0]
        df_idx += 1
        question['Documents'][df_idx] = document

    random_dataset.append(question)
    # print("Terminato idx:", idx)

print(len(random_dataset))

72209


In [54]:
random_dataset_df = pl.DataFrame(random_dataset)
path = f"data/random_dataset_gold_at_{gold_document_position}.parquet"
random_dataset_df.write_parquet(path)